In [1]:
# Cell 0: The Definitive Installation Command

# This command explicitly targets and forcefully upgrades all core LangChain components,
# including the problematic 'langchain-community' package. This ensures that all
# related packages are brought to their latest, mutually compatible versions,
# resolving the final dependency conflict.

#!pip install -q --upgrade \
#    langchain \
#    langchain-core \
#    langchain-community \
#    langgraph \
#    langsmith \
#    langchain-openai \
#    dateparser \
#    pytz \
##    google-api-python-client \
 #   google-auth-oauthlib \
 #   requests

#print("\n✅ All dependencies have been forcefully upgraded to their latest stable versions.")
#print("   This should resolve all dependency conflicts.")
#print("‼️ IMPORTANT: Please restart the Jupyter kernel now before proceeding to the next cell.")
#print("   (Go to the 'Kernel' menu -> 'Restart Kernel...')")

In [2]:
# Cell 1: Imports (Corrected)
# --- Core Libraries ---
import json
import os
from datetime import datetime, timedelta
from threading import Thread
import logging

# --- Web Server ---
from flask import Flask, request, jsonify

# --- Data Validation & Time Handling ---
from pydantic import BaseModel, Field
from typing import List, Dict, Optional, Any
import pytz
from dateutil.parser import parse as date_parse

# --- Google Calendar API ---
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# --- LangChain & LangGraph for Agentic Workflow ---
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate
# The problematic line that caused the warning has been removed.

from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# --- Suppress unnecessary warnings ---
logging.basicConfig(level=logging.INFO)
logging.getLogger('googleapiclient.discovery_cache').setLevel(logging.ERROR)

print("Cell 1/13: All libraries imported successfully (Warning resolved).")

Cell 1/13: All libraries imported successfully (Warning resolved).


In [3]:
# Cell 2: LLM and Environment Configuration (Final Path Correction)

# --- vLLM Server Details for Llama 3.1 ---
# We are now pointing to the Llama 3.1 server on port 4000.
BASE_URL = "http://localhost:4000/v1"

# CRITICAL FIX: The model path MUST match the path used to start the vLLM server.
# The server was started with the RELATIVE path, not the absolute /home/user path.
MODEL_PATH = "Models/meta-llama/Meta-Llama-3.1-8B-Instruct"

API_KEY = "not-needed"

# --- Initialize the LLM client for LangChain ---
llm = ChatOpenAI(
    model=MODEL_PATH,
    api_key=API_KEY,
    base_url=BASE_URL,
    max_tokens=500,
)

# --- Define working hours and other constants ---
WORK_DAY_START_HOUR = 9
WORK_DAY_END_HOUR = 18

print("Cell 2/13: LLM client configured for Llama 3.1 with CORRECT model path.")

Cell 2/13: LLM client configured for Llama 3.1 with CORRECT model path.


In [4]:
# Cell 3: Pydantic State and Data Models (Timezone Enabled)

class ExtractedInfo(BaseModel):
    """Holds the structured information extracted from the email content by the LLM."""
    meeting_summary: str = Field(description="A brief summary or title for the meeting, extracted from the email content.")
    duration_minutes: int = Field(description="The duration of the meeting in minutes.")
    time_constraint: str = Field(description="The specific time constraint mentioned, e.g., 'next Thursday at 4pm', 'tomorrow morning', 'Monday at 11:00 A.M'.")
    
class Attendee(BaseModel):
    """Represents a single attendee and their status, now with timezone info."""
    email: str
    timezone: str = "UTC"  # Default to UTC if not specified
    events: List[Dict[str, Any]] = Field(default_factory=list)

class ProposedSlot(BaseModel):
    """Represents a concrete time slot for the meeting."""
    start: datetime
    end: datetime

class AgentState(BaseModel):
    """The main state object for our LangGraph agent."""
    request_data: Dict[str, Any]
    attendees: Optional[List[Attendee]] = Field(default_factory=list)
    organizer: Optional[str] = None
    all_attendee_emails: Optional[List[str]] = Field(default_factory=list)
    extracted_info: Optional[ExtractedInfo] = None
    proposed_slot: Optional[ProposedSlot] = None
    status_message: str = "Processing..."
    is_conflict: bool = False
    alternative_slots: List[ProposedSlot] = Field(default_factory=list)
    final_output: Optional[Dict[str, Any]] = None

    class Config:
        arbitrary_types_allowed = True

print("Cell 3/13: Pydantic models UPGRADED with timezone support.")

Cell 3/13: Pydantic models UPGRADED with timezone support.


In [5]:
# Cell 4: Agent Tools (Final Version with Resilient JSON Parsing)

# --- SIMULATED USER TIMEZONE DATABASE ---
USER_TIMEZONES = {
    "userone.amd@gmail.com": "Asia/Kolkata",
    "usertwo.amd@gmail.com": "America/Los_Angeles",
    "userthree.amd@gmail.com": "Europe/London",
    "teamadmin.amd@gmail.com": "Asia/Kolkata"
}
# ---

@tool
def extract_meeting_details_from_email(email_content: str, request_datetime_str: str) -> ExtractedInfo:
    """
    [FINAL ROBUST VERSION]
    Uses the LLM to parse email content. It now aggressively finds and extracts
    the JSON block from the LLM's potentially messy response string.
    """
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", """You are an expert assistant for parsing meeting requests from emails.
Your task is to extract the meeting summary, duration in minutes, and the specific time constraint.
The current date and time for context is {request_datetime}.
Analyze the user's text to determine these details accurately.

You MUST return ONLY a valid JSON object string. Do not add any other text or explanations.
Example format:
{{
  "meeting_summary": "Discussion about Project X",
  "duration_minutes": 60,
  "time_constraint": "next Monday at 2 PM"
}}"""),
        ("human", "Here is the email content:\n\n---\n{email_content}\n---")
    ])
    
    chain = prompt_template | llm
    
    try:
        response = chain.invoke({
            "email_content": email_content,
            "request_datetime": request_datetime_str
        })
        
        response_content = response.content
        
        # --- Start of New Resilient Parsing Logic ---
        # Find the first '{' and the last '}' to isolate the JSON object
        json_start = response_content.find('{')
        json_end = response_content.rfind('}')
        
        if json_start != -1 and json_end != -1:
            json_string = response_content[json_start:json_end+1]
            parsed_json = json.loads(json_string)
            return ExtractedInfo(**parsed_json)
        else:
            # If no JSON object is found, raise an error
            raise ValueError("No JSON object found in the LLM response.")
        # --- End of New Resilient Parsing Logic ---
        
    except (json.JSONDecodeError, TypeError, AttributeError, ValueError) as e:
        logging.error(f"LLM parsing failed! Error: {e}. Raw LLM output: {response.content if 'response' in locals() else 'No response'}")
        raise ValueError("LLM failed to return a valid JSON object.")


@tool
def get_calendar_events(attendee_email: str, start_time: datetime, end_time: datetime) -> List[Dict[str, Any]]:
    """Fetches events and correctly handles personal events."""
    # ... (This function is already correct and does not need changes)
    events_list = []
    token_path = f"Keys/{attendee_email.split('@')[0]}.token"
    if not os.path.exists(token_path): return []
    try:
        creds = Credentials.from_authorized_user_file(token_path)
        service = build("calendar", "v3", credentials=creds)
        events_result = service.events().list(calendarId='primary', timeMin=start_time.isoformat(), timeMax=end_time.isoformat(), singleEvents=True, orderBy='startTime').execute()
        for event in events_result.get('items', []):
            start = date_parse(event['start'].get('dateTime', event['start'].get('date')))
            end = date_parse(event['end'].get('dateTime', event['end'].get('date')))
            try:
                event_attendees = event.get('attendees', [])
                if event_attendees:
                    attendee_list = [att['email'] for att in event_attendees]; num_attendees = len(attendee_list)
                else:
                    num_attendees = 1; attendee_list = [attendee_email]
            except KeyError:
                num_attendees = 1; attendee_list = [attendee_email]
            events_list.append({"summary": event.get("summary", "No Title"), "start": start, "end": end, "NumAttendees": num_attendees, "Attendees": attendee_list})
        return events_list
    except Exception as e:
        logging.error(f"Error fetching calendar for {attendee_email}: {e}")
        return []

@tool
def find_available_slots(attendees: List[Attendee], busy_slots: List[Dict[str, Any]], search_start_utc: datetime, duration_minutes: int) -> List[ProposedSlot]:
    """Finds common available slots within the overlapping working hours of all attendees."""
    # ... (This function is already correct and does not need changes)
    utc_tz = pytz.UTC
    golden_start, golden_end = utc_tz.localize(datetime.min), utc_tz.localize(datetime.max)
    for attendee in attendees:
        local_tz = pytz.timezone(attendee.timezone)
        local_workday_start = search_start_utc.astimezone(local_tz).replace(hour=WORK_DAY_START_HOUR, minute=0, second=0, microsecond=0)
        local_workday_end = search_start_utc.astimezone(local_tz).replace(hour=WORK_DAY_END_HOUR, minute=0, second=0, microsecond=0)
        utc_workday_start, utc_workday_end = local_workday_start.astimezone(utc_tz), local_workday_end.astimezone(utc_tz)
        golden_start, golden_end = max(golden_start, utc_workday_start), min(golden_end, utc_workday_end)
    if golden_start >= golden_end: return []
    utc_busy_intervals = sorted([ (event['start'].astimezone(utc_tz), event['end'].astimezone(utc_tz)) for event in busy_slots ])
    if not utc_busy_intervals: merged = []
    else:
        merged = [utc_busy_intervals[0]]
        for current_start, current_end in utc_busy_intervals[1:]:
            last_start, last_end = merged[-1]
            if current_start < last_end: merged[-1] = (last_start, max(last_end, current_end))
            else: merged.append((current_start, current_end))
    free_slots = []
    current_time = golden_start
    for busy_start, busy_end in merged:
        if current_time < busy_start: free_slots.append((current_time, busy_start))
        current_time = max(current_time, busy_end)
    if current_time < golden_end: free_slots.append((current_time, golden_end))
    meeting_duration = timedelta(minutes=duration_minutes)
    potential_slots = []
    for free_start, free_end in free_slots:
        slot_start = free_start
        while slot_start + meeting_duration <= free_end:
            potential_slots.append(ProposedSlot(start=slot_start, end=slot_start + meeting_duration))
            slot_start += timedelta(minutes=15)
            if len(potential_slots) >= 5: break
        if len(potential_slots) >= 5: break
    return potential_slots

print("Cell 4/13: Agent tools FINALIZED with resilient JSON parsing.")

Cell 4/13: Agent tools FINALIZED with resilient JSON parsing.


In [6]:
# Cell 5: Graph Nodes (Final Version for Timezone Logic)

def get_ist_timezone(): return pytz.timezone('Asia/Kolkata')
def get_utc_timezone(): return pytz.UTC

def initialize_state(state: AgentState) -> AgentState:
    """Node 1: Initializes the state and injects user timezones."""
    request = state.request_data
    organizer = request['From']
    attendee_emails = [att['email'] for att in request['Attendees']]
    all_emails = list(set([organizer] + attendee_emails))
    
    # Inject timezone information from our simulated database
    state.attendees = [Attendee(email=email, timezone=USER_TIMEZONES.get(email, "UTC")) for email in all_emails]
    
    state.organizer = organizer
    state.all_attendee_emails = all_emails
    state.status_message = "State initialized. Extracting meeting details..."
    logging.info(f"Initialized state for Request ID: {request['Request_id']}")
    return state

def extract_details(state: AgentState) -> AgentState:
    # ... (This function is unchanged)
    request = state.request_data
    logging.info("Extracting details using LLM...")
    try:
        extracted_info = extract_meeting_details_from_email.invoke({"email_content": request['EmailContent'], "request_datetime_str": request['Datetime']})
        state.extracted_info = extracted_info
        state.status_message = f"Extracted details: {extracted_info.model_dump_json()}"
    except ValueError as e:
        state.status_message = "ERROR: LLM failed to extract details."
        state.extracted_info = None
    return state

def handle_llm_failure(state: AgentState) -> AgentState:
    # ... (This function is unchanged)
    state.final_output = {"error": "Could not proceed due to LLM failure."}
    return state

def determine_initial_slot(state: AgentState) -> AgentState:
    """Node 3: Converts the LLM's time constraint into a UTC datetime object."""
    logging.info("Determining initial proposed slot...")
    from dateparser import search
    
    request_dt_str = state.request_data['Datetime']
    organizer_tz_str = USER_TIMEZONES.get(state.organizer, "UTC")
    organizer_tz = pytz.timezone(organizer_tz_str)
    
    # The time is parsed relative to the ORGANIZER's timezone.
    search_results = search.search_dates(state.extracted_info.time_constraint, settings={'PREFER_DATES_FROM': 'future', 'RELATIVE_BASE': date_parse(request_dt_str).astimezone(organizer_tz)})
    if not search_results:
        # ... (error handling is unchanged)
        return state

    start_time_local = search_results[0][1]
    if start_time_local.tzinfo is None: start_time_local = organizer_tz.localize(start_time_local)
    
    # Convert the proposed slot to UTC for all future logic
    start_time_utc = start_time_local.astimezone(get_utc_timezone())
    end_time_utc = start_time_utc + timedelta(minutes=state.extracted_info.duration_minutes)
    
    state.proposed_slot = ProposedSlot(start=start_time_utc, end=end_time_utc)
    state.status_message = f"Initial proposed slot (UTC): {start_time_utc.isoformat()} to {end_time_utc.isoformat()}"
    logging.info(state.status_message)
    return state

def check_availability(state: AgentState) -> AgentState:
    """Node 4: Checks the availability of all attendees for the proposed UTC slot."""
    logging.info("Checking attendee availability...")
    total_conflicts = 0
    for attendee in state.attendees:
        # Define search window for the entire day in the attendee's local time
        day_start_local = state.proposed_slot.start.astimezone(pytz.timezone(attendee.timezone)).replace(hour=0, minute=0, second=0)
        day_end_local = day_start_local + timedelta(days=1)
        
        attendee.events = get_calendar_events.invoke({"attendee_email": attendee.email, "start_time": day_start_local, "end_time": day_end_local})
        
        for event in attendee.events:
            # All comparisons must happen in UTC
            event_start_utc = event['start'].astimezone(get_utc_timezone())
            event_end_utc = event['end'].astimezone(get_utc_timezone())
            if state.proposed_slot.start < event_end_utc and state.proposed_slot.end > event_start_utc:
                total_conflicts += 1
                logging.warning(f"CONFLICT for {attendee.email}: Busy with '{event['summary']}'")
                break
    
    if total_conflicts > 0: state.is_conflict = True; state.status_message = f"Found {total_conflicts} conflict(s)."
    else: state.is_conflict = False; state.status_message = "All attendees are available at the proposed time."
    logging.info(state.status_message)
    return state

def find_alternatives(state: AgentState) -> AgentState:
    """Node 5: Finds alternative slots using the new timezone-aware tool."""
    logging.info("Conflict detected. Finding alternative slots...")
    all_events = [event for attendee in state.attendees for event in attendee.events]
    
    alternative_slots = find_available_slots.invoke({
        "attendees": state.attendees,
        "busy_slots": all_events,
        "search_start_utc": state.proposed_slot.start,
        "duration_minutes": state.extracted_info.duration_minutes
    })
    
    state.alternative_slots = alternative_slots
    if alternative_slots:
        state.proposed_slot = alternative_slots[0]
        state.status_message = f"Found alternatives. Proposing new UTC slot: {alternative_slots[0].start.isoformat()}"
    else:
        state.status_message = "Conflict detected, but no overlapping working hours or free slots were found."
    logging.info(state.status_message)
    return state

def format_final_response(state: AgentState) -> AgentState:
    """Node 6: Formats the final UTC time back into the required output format."""
    logging.info("Formatting the final response to match specification...")
    request = state.request_data
    proposed_slot = state.proposed_slot
    
    final_json = {"Request_id": request["Request_id"], "Datetime": request["Datetime"], "Location": request["Location"], "From": request["From"], "Attendees": [], "Subject": state.extracted_info.meeting_summary, "EmailContent": request["EmailContent"], "EventStart": None, "EventEnd": None, "Duration_mins": str(state.extracted_info.duration_minutes), "MetaData": {}}
    
    if proposed_slot:
        # Display the final time in the organizer's local timezone for clarity
        organizer_tz = pytz.timezone(USER_TIMEZONES.get(state.organizer, "UTC"))
        final_json["EventStart"] = proposed_slot.start.astimezone(organizer_tz).isoformat()
        final_json["EventEnd"] = proposed_slot.end.astimezone(organizer_tz).isoformat()

    for attendee_state in state.attendees:
        attendee_output = {"email": attendee_state.email, "events": []}
        for event in attendee_state.events:
            attendee_output["events"].append({"StartTime": event['start'].isoformat(), "EndTime": event['end'].isoformat(), "Summary": event['summary'], "NumAttendees": event.get('NumAttendees', "N/A"), "Attendees": event.get('Attendees', "N/A")})
        final_json["Attendees"].append(attendee_output)
        
    state.final_output = final_json
    logging.info("Final response formatted.")
    return state

print("Cell 5/13: Graph nodes UPGRADED for multi-timezone scheduling.")

Cell 5/13: Graph nodes UPGRADED for multi-timezone scheduling.


In [7]:
# Cell 6: Building the LangGraph Workflow (Corrected for Error Handling)

def decide_after_extraction(state: AgentState) -> str:
    """Decides where to go after the LLM extraction step."""
    if state.extracted_info is None:
        return "handle_llm_failure"
    else:
        return "determine_slot"

def decide_on_conflict(state: AgentState) -> str:
    """Decides where to go after checking calendars."""
    if state.is_conflict:
        return "find_alternatives"
    else:
        return "format_final_response"

workflow = StateGraph(AgentState)

workflow.add_node("initialize", initialize_state)
workflow.add_node("extract_details", extract_details)
workflow.add_node("determine_slot", determine_initial_slot)
workflow.add_node("check_availability", check_availability)
workflow.add_node("find_alternatives", find_alternatives)
workflow.add_node("format_final_response", format_final_response)
workflow.add_node("handle_llm_failure", handle_llm_failure) # New error node

workflow.set_entry_point("initialize")
workflow.add_edge("initialize", "extract_details")

# New conditional edge after extraction
workflow.add_conditional_edges(
    "extract_details",
    decide_after_extraction,
    {
        "determine_slot": "determine_slot",
        "handle_llm_failure": "handle_llm_failure"
    }
)
# The error path goes directly to the end
workflow.add_edge("handle_llm_failure", END)

workflow.add_edge("determine_slot", "check_availability")
workflow.add_conditional_edges("check_availability", decide_on_conflict, {"find_alternatives": "find_alternatives", "format_final_response": "format_final_response"})
workflow.add_edge("find_alternatives", "format_final_response")
workflow.add_edge("format_final_response", END)

memory = MemorySaver()
agent_runnable = workflow.compile(checkpointer=memory)

print("Cell 6/13: LangGraph workflow updated with robust error path.")


Cell 6/13: LangGraph workflow updated with robust error path.


In [8]:
# Cell 7: Flask Web Server Setup (Final Version for Exact Output Match)

app = Flask(__name__)
request_history = []

def your_meeting_assistant(data: Dict[str, Any]) -> Dict[str, Any]:
    """
    This is the main function that processes an incoming meeting request.
    It uses the compiled LangGraph agent to perform all the work.
    This function still returns the full object with 'processed' and 'output'
    as required by one part of the spec.
    """
    logging.info(f"\n\n--- New Request Received: {data['Request_id']} ---")
    initial_state = {"request_data": data}
    config = {"configurable": {"thread_id": data['Request_id']}}
    
    final_state_snapshot = agent_runnable.invoke(initial_state, config=config)
    
    final_output = final_state_snapshot.get('final_output')
    
    processed_data = {}
    extracted_info_obj = final_state_snapshot.get('extracted_info')
    if extracted_info_obj:
        processed_data["extracted_info"] = extracted_info_obj.model_dump()
    else:
        processed_data["error"] = "LLM failed to extract details."

    processed_data["agent_status_message"] = final_state_snapshot.get('status_message', 'Done.')
    
    response_data = data.copy()
    response_data["processed"] = processed_data
    response_data["output"] = final_output
    
    logging.info(f"--- Request {data['Request_id']} Processed ---")
    return response_data

@app.route('/receive', methods=['POST'])
def receive():
    """
    The main API endpoint that receives scheduling requests.
    CRITICAL CHANGE: This endpoint now returns ONLY the 'output' dictionary
    to exactly match the final graded output specification.
    """
    data = request.get_json()
    if not data or "Request_id" not in data:
        return jsonify({"error": "Invalid JSON data"}), 400
    print(f"\n[Flask] Received request: {json.dumps(data, indent=2)}")
    
    # Run the full agent
    full_response = your_meeting_assistant(data)
    
    # Store the full response for our own debugging/logging
    request_history.append(full_response)
    
    # Extract only the 'output' part for the final response to the judges
    final_output_json = full_response.get('output', {"error": "Agent failed to produce an output."})
    
    print(f"\n[Flask] Sending final graded response: {json.dumps(final_output_json, indent=2)}")
    return jsonify(final_output_json)

def run_flask():
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

print("Cell 7/13: Flask application finalized for exact output match.")

Cell 7/13: Flask application finalized for exact output match.


In [9]:
# Cell 8: Start the Flask Server

# We run the Flask app in a separate thread to keep the notebook interactive.
# The 'daemon=True' flag ensures the thread will close when the main notebook process ends.
flask_thread = Thread(target=run_flask, daemon=True)
flask_thread.start()

print("Cell 8/13: Flask server starting in a background thread...")
print("The server is now listening on http://0.0.0.0:5000/receive")

Cell 8/13: Flask server starting in a background thread...
The server is now listening on http://0.0.0.0:5000/receive
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://129.212.176.242:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:root:

--- New Request Received: timezone-test-india-us-uk-001 ---
INFO:root:Initialized state for Request ID: timezone-test-india-us-uk-001
INFO:root:Extracting details using LLM...



[Flask] Received request: {
  "Request_id": "timezone-test-india-us-uk-001",
  "Datetime": "2025-07-21T10:00:00Z",
  "Location": "Virtual / Cross-Timezone",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "usertwo.amd@gmail.com"
    },
    {
      "email": "userthree.amd@gmail.com"
    }
  ],
  "Subject": "Global Sync for Q3 Planning",
  "EmailContent": "Hi team, let's connect for our Q3 planning session. Can we meet for an hour sometime tomorrow morning?"
}


INFO:httpx:HTTP Request: POST http://localhost:4000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Determining initial proposed slot...
INFO:root:Initial proposed slot (UTC): 2025-07-22T10:00:00+00:00 to 2025-07-22T11:00:00+00:00
INFO:root:Checking attendee availability...
INFO:root:Found 1 conflict(s).
INFO:root:Conflict detected. Finding alternative slots...
INFO:root:Conflict detected, but no overlapping working hours or free slots were found.
INFO:root:Formatting the final response to match specification...
INFO:root:Final response formatted.
INFO:root:--- Request timezone-test-india-us-uk-001 Processed ---
INFO:werkzeug:127.0.0.1 - - [13/Jul/2025 05:01:22] "POST /receive HTTP/1.1" 200 -



[Flask] Sending final graded response: {
  "Request_id": "timezone-test-india-us-uk-001",
  "Datetime": "2025-07-21T10:00:00Z",
  "Location": "Virtual / Cross-Timezone",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "userone.amd@gmail.com",
      "events": [
        {
          "StartTime": "2025-07-21T18:00:00+05:30",
          "EndTime": "2025-07-22T09:00:00+05:30",
          "Summary": "Off Hours",
          "NumAttendees": 1,
          "Attendees": [
            "userone.amd@gmail.com"
          ]
        },
        {
          "StartTime": "2025-07-22T18:00:00+05:30",
          "EndTime": "2025-07-23T09:00:00+05:30",
          "Summary": "Off Hours",
          "NumAttendees": 1,
          "Attendees": [
            "userone.amd@gmail.com"
          ]
        }
      ]
    },
    {
      "email": "userthree.amd@gmail.com",
      "events": [
        {
          "StartTime": "2025-07-21T16:00:00+05:30",
          "EndTime": "2025-07-22T07:30:00+05:30",
   

...


[Flask] Sending final graded response: {
  "Request_id": "TC2-USERTHREE-BUSY-6118b54f",
  "Datetime": "2025-07-02T12:34:55",
  "Location": "IIT Mumbai",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "userone.amd@gmail.com",
      "events": [
        {
          "StartTime": "2025-07-06T18:00:00+05:30",
          "EndTime": "2025-07-07T09:00:00+05:30",
          "Summary": "Off Hours",
          "NumAttendees": 1,
          "Attendees": [
            "userone.amd@gmail.com"
          ]
        },
        {
          "StartTime": "2025-07-07T18:00:00+05:30",
          "EndTime": "2025-07-08T09:00:00+05:30",
          "Summary": "Off Hours",
          "NumAttendees": 1,
          "Attendees": [
            "userone.amd@gmail.com"
          ]
        }
      ]
    },
    {
      "email": "userthree.amd@gmail.com",
      "events": [
        {
          "StartTime": "2025-07-06T16:00:00+05:30",
          "EndTime": "2025-07-07T07:30:00+05:30",
          "Summary":

INFO:httpx:HTTP Request: POST http://localhost:4000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Determining initial proposed slot...
INFO:root:Initial proposed slot (UTC): 2026-01-02T05:30:00+00:00 to 2026-01-02T06:30:00+00:00
INFO:root:Checking attendee availability...
INFO:root:All attendees are available at the proposed time.
INFO:root:Formatting the final response to match specification...
INFO:root:Final response formatted.
INFO:root:--- Request TC3-ALL-BUSY-6118b54f Processed ---
INFO:werkzeug:127.0.0.1 - - [13/Jul/2025 05:15:12] "POST /receive HTTP/1.1" 200 -
INFO:root:

--- New Request Received: TC4-USERTHREE-BUSY-AGAIN-6118b54f ---
INFO:root:Initialized state for Request ID: TC4-USERTHREE-BUSY-AGAIN-6118b54f
INFO:root:Extracting details using LLM...



[Flask] Sending final graded response: {
  "Request_id": "TC3-ALL-BUSY-6118b54f",
  "Datetime": "2025-07-02T12:34:55",
  "Location": "IIT Mumbai",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "userone.amd@gmail.com",
      "events": [
        {
          "StartTime": "2026-01-01T18:00:00+05:30",
          "EndTime": "2026-01-02T09:00:00+05:30",
          "Summary": "Off Hours",
          "NumAttendees": 1,
          "Attendees": [
            "userone.amd@gmail.com"
          ]
        },
        {
          "StartTime": "2026-01-02T18:00:00+05:30",
          "EndTime": "2026-01-03T09:00:00+05:30",
          "Summary": "Off Hours",
          "NumAttendees": 1,
          "Attendees": [
            "userone.amd@gmail.com"
          ]
        }
      ]
    },
    {
      "email": "userthree.amd@gmail.com",
      "events": [
        {
          "StartTime": "2026-01-01T16:00:00+05:30",
          "EndTime": "2026-01-02T07:30:00+05:30",
          "Summary": "Off 

INFO:httpx:HTTP Request: POST http://localhost:4000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Determining initial proposed slot...
INFO:root:Initial proposed slot (UTC): 2026-01-02T04:30:00+00:00 to 2026-01-02T05:15:00+00:00
INFO:root:Checking attendee availability...
INFO:root:All attendees are available at the proposed time.
INFO:root:Formatting the final response to match specification...
INFO:root:Final response formatted.
INFO:root:--- Request TC4-USERTHREE-BUSY-AGAIN-6118b54f Processed ---
INFO:werkzeug:127.0.0.1 - - [13/Jul/2025 05:15:13] "POST /receive HTTP/1.1" 200 -



[Flask] Sending final graded response: {
  "Request_id": "TC4-USERTHREE-BUSY-AGAIN-6118b54f",
  "Datetime": "2025-07-02T12:34:55",
  "Location": "IIT Mumbai",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "userone.amd@gmail.com",
      "events": [
        {
          "StartTime": "2026-01-01T18:00:00+05:30",
          "EndTime": "2026-01-02T09:00:00+05:30",
          "Summary": "Off Hours",
          "NumAttendees": 1,
          "Attendees": [
            "userone.amd@gmail.com"
          ]
        },
        {
          "StartTime": "2026-01-02T18:00:00+05:30",
          "EndTime": "2026-01-03T09:00:00+05:30",
          "Summary": "Off Hours",
          "NumAttendees": 1,
          "Attendees": [
            "userone.amd@gmail.com"
          ]
        }
      ]
    },
    {
      "email": "userthree.amd@gmail.com",
      "events": [
        {
          "StartTime": "2026-01-01T16:00:00+05:30",
          "EndTime": "2026-01-02T07:30:00+05:30",
          "Sum

In [10]:
# Cell 15: Advanced Multi-Timezone API Test

import requests
import json

# The sample JSON data designed to test multi-timezone scheduling
timezone_test_data = {
    "Request_id": "timezone-test-india-us-uk-001",
    "Datetime": "2025-07-21T10:00:00Z",
    "Location": "Virtual / Cross-Timezone",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {"email": "usertwo.amd@gmail.com"},
        {"email": "userthree.amd@gmail.com"}
    ],
    "Subject": "Global Sync for Q3 Planning",
    "EmailContent": "Hi team, let's connect for our Q3 planning session. Can we meet for an hour sometime tomorrow morning?"
}

# The URL of your running Flask server
server_url = "http://localhost:5000/receive"
headers = {"Content-Type": "application/json"}

print("--- Sending Multi-Timezone Test Request ---")
response = requests.post(server_url, headers=headers, data=json.dumps(timezone_test_data))

print(f"\n--- Server Response (Status Code: {response.status_code}) ---")
if response.status_code == 200:
    # Print the final output JSON beautifully
    print(json.dumps(response.json(), indent=2))
else:
    print("An error occurred:")
    print(response.text)

--- Sending Multi-Timezone Test Request ---

--- Server Response (Status Code: 200) ---
{
  "Attendees": [
    {
      "email": "userone.amd@gmail.com",
      "events": [
        {
          "Attendees": [
            "userone.amd@gmail.com"
          ],
          "EndTime": "2025-07-22T09:00:00+05:30",
          "NumAttendees": 1,
          "StartTime": "2025-07-21T18:00:00+05:30",
          "Summary": "Off Hours"
        },
        {
          "Attendees": [
            "userone.amd@gmail.com"
          ],
          "EndTime": "2025-07-23T09:00:00+05:30",
          "NumAttendees": 1,
          "StartTime": "2025-07-22T18:00:00+05:30",
          "Summary": "Off Hours"
        }
      ]
    },
    {
      "email": "userthree.amd@gmail.com",
      "events": [
        {
          "Attendees": [
            "userthree.amd@gmail.com"
          ],
          "EndTime": "2025-07-22T07:30:00+05:30",
          "NumAttendees": 1,
          "StartTime": "2025-07-21T16:00:00+05:30",
          "Sum

In [12]:
# Cell 14: Comprehensive Test Suite (Final Corrected Version)

import requests
import json
import time

# --- Define all test cases ---

# Test Case 1: All attendees are expected to be available.
test_case_1 = {
    "Request_id": "TC1-BOTH-AVAILABLE-6118b54f",
    "Datetime": "2025-07-02T12:34:55",
    "Location": "IIT Mumbai",
    "From": "userone.amd@gmail.com",
    "Attendees": [{"email": "usertwo.amd@gmail.com"}, {"email": "userthree.amd@gmail.com"}],
    "Subject": "Goals Discussion",
    "EmailContent": "Hi Team. Let's meet next Thursday for 30 minutes and discuss about our Goals."
}

# Test Case 2: USERTHREE is busy, USERONE and USERTWO are free.
test_case_2 = {
    "Request_id": "TC2-USERTHREE-BUSY-6118b54f",
    "Datetime": "2025-07-02T12:34:55",
    "Location": "IIT Mumbai",
    "From": "userone.amd@gmail.com",
    "Attendees": [{"email": "usertwo.amd@gmail.com"}, {"email": "userthree.amd@gmail.com"}],
    "Subject": "Client Validation - Urgent",
    "EmailContent": "Hi Team. We’ve just received quick feedback from the client. Let’s prioritize resolving this promptly. Let’s meet Monday at 9:00 AM for 30 minutes to discuss and resolve this issue."
}

# Test Case 3: All attendees are busy.
test_case_3 = {
    "Request_id": "TC3-ALL-BUSY-6118b54f",
    "Datetime": "2025-07-02T12:34:55",
    "Location": "IIT Mumbai",
    "From": "userone.amd@gmail.com",
    "Attendees": [{"email": "usertwo.amd@gmail.com"}, {"email": "userthree.amd@gmail.com"}],
    "Subject": "Project Status",
    "EmailContent": "Hi Team. Let's meet on Tuesday at 11:00 A.M for an hour and discuss about our on-going Projects."
}

# Test Case 4: USERTHREE is busy (again, different scenario).
test_case_4 = {
    "Request_id": "TC4-USERTHREE-BUSY-AGAIN-6118b54f",
    "Datetime": "2025-07-02T12:34:55",
    "Location": "IIT Mumbai",
    "From": "userone.amd@gmail.com",
    "Attendees": [{"email": "usertwo.amd@gmail.com"}, {"email": "userthree.amd@gmail.com"}],
    "Subject": "Client Feedback",
    "EmailContent": "Hi Team. We’ve received the final feedback from the client. Let’s meet on Wednesday at 10:00 A.M. for 45 minutes."
}

all_test_cases = [test_case_1, test_case_2, test_case_3, test_case_4]
server_url = "http://localhost:5000/receive"
headers = {"Content-Type": "application/json"}

# --- Iterate through and run all test cases ---

# Give the server a moment to start up if it was just launched
time.sleep(2) 

for i, test_case in enumerate(all_test_cases, 1):
    print(f"\n\n{'='*20} Sending Test Case {i} ({test_case['Request_id']}) {'='*20}")
    
    response = requests.post(server_url, headers=headers, data=json.dumps(test_case))
    
    if response.status_code == 200:
        print(f"\n--- Final Output for Test Case {i} ---")
        response_json = response.json()
        
        # CRITICAL FIX: The response_json *is* the final output. We print it directly.
        print(json.dumps(response_json, indent=2))
        
    else:
        print(f"ERROR on Test Case {i}: Server returned status {response.status_code}")
        print(response.text)

print(f"\n\n{'='*20} COMPREHENSIVE TEST SUITE COMPLETE {'='*20}")



==================== Sending Test Case 1 (TC1-BOTH-AVAILABLE-6118b54f) ====================

--- Final Output for Test Case 1 ---
{
  "Attendees": [
    {
      "email": "userone.amd@gmail.com",
      "events": [
        {
          "Attendees": [
            "userone.amd@gmail.com"
          ],
          "EndTime": "2025-07-03T09:00:00+05:30",
          "NumAttendees": 1,
          "StartTime": "2025-07-02T18:00:00+05:30",
          "Summary": "Off Hours"
        },
        {
          "Attendees": [
            "userone.amd@gmail.com"
          ],
          "EndTime": "2025-07-04T09:00:00+05:30",
          "NumAttendees": 1,
          "StartTime": "2025-07-03T18:00:00+05:30",
          "Summary": "Off Hours"
        }
      ]
    },
    {
      "email": "userthree.amd@gmail.com",
      "events": [
        {
          "Attendees": [
            "userthree.amd@gmail.com"
          ],
          "EndTime": "2025-07-02T07:30:00+05:30",
          "NumAttendees": 1,
          "StartTime":